### Import Libs

In [76]:
import matplotlib.pyplot as plt
import random
from scipy import special as sc
import itertools

### Create Individual

In [77]:
def create_individual():
    # TODO: create a random chromosome
    return [random.randrange(8) for idx in range(8)]

### Generate Population

In [78]:
def generate_population(population_size:int = 20):
    population = []

    for _ in range(population_size):
        new_individual = create_individual()
        population.append(new_individual)

    return population

### Parent Selection

In [79]:
def parent_selection(population):
    parents = []
    #TODO: choose the pairs of parents
    for ind in population:
        #select parents with probability proportional to their fitness score
        if random.randrange(sc.comb(8, 2)*2) < fitness_score(ind):
            parents.append(ind)
    return parents

### Next Generation Selection

In [80]:
def next_generation_selection(children_population):
    next_generation = []
    #TODO: choose the next_generation
    #select individuals to become parents
    parents = parent_selection(population)

    #recombination. Create new offsprings
    offsprings = crossover(parents)

    #mutation
    offsprings = list(map(mutate, offsprings))

    #introduce top-scoring individuals from previous generation and keep top fitness individuals
    next_generation = offsprings

    for ind in population:
        next_generation.append(ind)

    next_generation = sorted(next_generation, key=lambda ind: fitness_score(ind), reverse=True)[:20]
    return next_generation

### Crossover

In [81]:
def crossover(parents):
    offsprings = []
    #TODO: implement you crossover algorithm
    cross_points = random.sample(range(8), 1)
    permutations = list(itertools.permutations(parents, 2))

    for perm in permutations:
        offspring = []

        #track starting index of sublist
        start_pt = 0

        for parent_idx, cross_point in enumerate(cross_points): #doesn't account for last parent

            #sublist of parent to be crossed
            parent_part = perm[parent_idx][start_pt:cross_point]
            offspring.append(parent_part)

            #update index pointer
            start_pt = cross_point

        #last parent
        last_parent = perm[-1]
        parent_part = last_parent[cross_point:]
        offspring.append(parent_part)

        #flatten the list since append works kinda differently
        offsprings.append(list(itertools.chain(*offspring)))
    return offsprings

### Mutation

In [82]:
def mutate(seq):
    #TODO: implement mutate
    for row in range(len(seq)):
        if random.random() < 0.05:
            seq[row] = random.randrange(8)

    return seq

### Score Function

In [93]:
def fitness_score(seq):
    fit_score = 0
    #TODO: calculate fitness score
    seq = list(seq)
    for row in range(8):
        col = seq[row]

        for other_row in range(8):

            #queens cannot pair with itself
            if other_row == row:
                continue
            if seq[other_row] == col:
                continue
            if other_row + seq[other_row] == row + col:
                continue
            if other_row - seq[other_row] == row - col:
                continue
            #score++ if every pair of queens are non-attacking.
            fit_score += 1

    return fit_score/2

## Calculate Population Fitness

In [84]:
def population_fitness(population):
    pop_fitness = 0
    individuals_fitnesses:dict = {}
    for chromosome in population:
        fitness_sc = fitness_score(chromosome)
        pop_fitness += fitness_sc
        individuals_fitnesses[tuple(chromosome)] = fitness_sc
    return individuals_fitnesses ,pop_fitness

### Termination Condition

In [85]:
def check_end(population) -> bool:
    #TODO: check end of algorithm
    for ind in population:
        score = fitness_score(ind)
        if score == sc.comb(8, 2):
            return True

    return False

### Run Your Algorithms

In [100]:
#The genetic algorithm
generation = 0
solution_is_found = False
fitness = []

population = generate_population()
population , pop_fitness = population_fitness(population)

generation = 0

while not check_end(population):
    # TODO: this is a default setup which you maybe need to change.
    population = parent_selection(population)
    population = crossover(population)
    population = mutate(population)
    individuals_fitnesses , pop_fitness = population_fitness(population)
    population = next_generation_selection(population)

    generation += 1
    print(population_fitness)
    print(population.__len__())
    avg_fit = population_fitness / population.__len__()
    fitness.append(avg_fit)


<ipython-input-79-7a4394c831bc>:6: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  if random.randrange(sc.comb(8, 2)*2) < fitness_score(ind):


TypeError: 'int' object is not iterable

### See Your Progress Over Generations

In [ ]:
plt.bar(range(generation), fitness, color='skyblue')
plt.xlabel('Generation', fontsize=20)
plt.ylabel('Average Fitness', fontsize=20)
plt.title('Fitness over Generations', fontsize=20)
plt.show()